In [ ]:
!pip install nltk==3.8.1

import nltk
from nltk.tokenize import sent_tokenize

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [ ]:
from sentence_transformers import util

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
# model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [ ]:
import json

filename = 'data (2).json'

with open(filename, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

In [ ]:
import re

def clean_text(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

def remove_empty_sentences(sentences):
    return [sent for sent in sentences if sent.strip()]

def extract_paragraphs(text):
    paragraphs = re.split('\n\s*\n', text)
    return list(filter(None, map(str.strip, paragraphs)))

def normalize_text(text):
    # Объединяем абзацы в сплошной текст
    normalized_text = re.sub(r'\n\s*\n', '. ', text)
    normalized_text = re.sub(r'\s+', ' ', normalized_text)
    return normalized_text.strip()

def split_into_meaningful_sections(normalized_text):
    sections = []
    sentences = sent_tokenize(' '.join(normalized_text))
    current_section = []
    threshold = 3

    for sentence in sentences:
        if len(current_section) >= threshold:
            sections.append(". ".join(current_section))
            current_section.clear()
        current_section.append(sentence)

    if current_section:
        sections.append(". ".join(current_section))

    return sections

<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1092589981.py:12: SyntaxWarning: invalid escape sequence '\s'
  paragraphs = re.split('\n\s*\n', text)  # Разделение по двум или более пустым строкам


In [ ]:
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id

texts = [d['text'] for d in dataset]

batch_size = 128

batches = [texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]

document_embeddings = []
for batch in tqdm(batches):
    batch_embeddings = embedding_model.encode(batch)
    document_embeddings.extend(batch_embeddings)

document_embeddings = np.array(document_embeddings)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss

faiss.normalize_L2(document_embeddings)

dim = document_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(document_embeddings)

In [ ]:
def retrieve_relevant_docs(query, k=5):
    query_embedding = embedding_model.encode([query])
    print('query_embedding')
    faiss.normalize_L2(query_embedding)
    print('faiss.normalize_L2')
    distances, indices = index.search(query_embedding, k)
    print('Дистанции и индексы нашел')

    relevant_docs = []
    for idx in indices[0]:
        doc = dataset[idx.item()]
        relevant_docs.append(doc)

    return relevant_docs

In [ ]:
def select_best_segment(relevant_docs, query):
    best_segments = []
    for doc in relevant_docs:
        paragraphs = extract_paragraphs(normalize_text(doc['text']))
        segments = split_into_meaningful_sections(paragraphs)
        if len(segments) > 0:
            encoded_query = embedding_model.encode([query])[0]
            encoded_segments = embedding_model.encode(segments)
            similarities = util.cos_sim(encoded_query, encoded_segments)
            best_segment_idx = np.argmax(similarities.squeeze())
            best_segments.append(segments[best_segment_idx])
    return best_segments

In [ ]:
def select_best_sentences(relevant_docs, query):
    best_sentences = []
    for doc in relevant_docs:
        sentences = sent_tokenize(doc['text'])
        print('Документ разделил')
        scores = [
            util.cos_sim(embedding_model.encode(query), embedding_model.encode(sent)).item()
            for sent in sentences
        ]
        print('Счет посчитал')
        best_sentence_idx = np.argmax(scores)
        best_sentences.append(sentences[best_sentence_idx])

    return best_sentences

In [ ]:
def select_best_sentences_fast(relevant_docs, query):
    best_sentences = []
    print(f'Кол-во документов для анализа: {len(relevant_docs)}')
    for doc in relevant_docs:
        sentences = sent_tokenize(doc['text'])
        print(sentences)
        print('Документ разделил')
        encoded_query = embedding_model.encode([query])[0]
        encoded_sentences = embedding_model.encode(sentences)
        print('Эмбеддинги закодировал')
        similarities = util.cos_sim(encoded_query, encoded_sentences)
        print('Похожесть посчитал')
        print(f'Похожести: {similarities}')
        best_sentence_idx = np.argmax(similarities)
        print(best_sentence_idx)
        print('Лучшее предложение отыскал')
        best_sentences.append(sentences[best_sentence_idx])

    return best_sentences

In [ ]:
def select_best_paragraph(relevant_docs, query):
    best_paragraphs = []
    for doc in relevant_docs:
        paragraphs = extract_paragraphs(doc['text'])
        print(paragraphs)
        if len(paragraphs) > 0:
            encoded_query = embedding_model.encode([query])[0]
            encoded_paragraphs = embedding_model.encode(paragraphs)
            similarities = util.cos_sim(encoded_query, encoded_paragraphs)
            best_paragraph_idx = np.argmax(similarities.squeeze())
            best_paragraphs.append(paragraphs[best_paragraph_idx])
    return best_paragraphs

In [ ]:
def rag_generate(query):
      relevant_docs = retrieve_relevant_docs(query)
      print('Документы подобрал')

      best_sentences = select_best_paragraph(relevant_docs, query)
      print('Предложения выбрал')
      print('Лучшие предложения:')
      print(best_sentences)

      context = ' '.join(list(set(best_sentences)))

      input_text = f"вопрос: {query}\nконтекст: {context}\n"

      inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

      outputs = model.generate(inputs.input_ids, max_new_tokens=512, do_sample=True, temperature=0.7)
      answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
      return answer.strip()

In [ ]:
query = "Цветы виды"

final_answer = rag_generate(query)

print(final_answer)

query_embedding
faiss.normalize_L2
Дистанции и индексы нашел
Документы подобрал
['Секреты выращивания домашних цветов\nИгорь Семенов', 'Поделился опытом выращивания в этом произведении. Про многие растения вы прочитаете впервые. Книга рассчитана на широкий круг читателей и цветоводов. Превратите свой дом в райский сад!', 'Секреты выращивания домашних цветов', 'Игорь Семенов', 'Художник – оформитель обложки Дмитрий Владимирович Чаринцев', '©\xa0Игорь Семенов,\xa02024', 'ISBN\xa0978-5-0064-0555-4', 'Создано в интеллектуальной издательской системе Ridero', 'СЕКРЕТЫ ВЫРАЩИВАНИЯ ДОМАШНИХ ЦВЕТОВ', 'О\xa0книге', 'Автор книги заядлый цветовод. В\xa0своей квартире вырастил двадцать домашних цветков. Собрал информацию и\xa0секреты по\xa0самым распространенным и\xa0экзотерическим цветам. Поделился опытом выращивания в\xa0этом произведении. Про многие растения вы прочитаете впервые. Книга рассчитана на\xa0широкий круг читателей и\xa0цветоводов. Превратите свой дом в\xa0райский\xa0сад!', 'Выращивае